You probably want to just use ``refit.py`` instead of this workbench script.

Before running, follow the installation instructions in the `AdversarialInfluenceWorkbench/examples/microcredit_mixture/README.md` file.

In [1]:
import paragami
import vittles
import autograd
from autograd import numpy as np
#from autograd import scipy as sp
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from copy import deepcopy
import os

import time
import scipy as osp

In [2]:
import microcredit_mixture_rgiordandev
from microcredit_mixture_rgiordandev import microcredit_vb_lib as mm_lib

In [3]:
from microcredit_mixture_rgiordandev import \
    get_log_prob, get_param_pattern, get_log_prob_with_jacobian

In [4]:
base_dir = os.path.join(
    '/home/rgiordan/Documents/git_repos/',
    'AdversarialInfluenceWorkbench/examples/microcredit_mixture/')

use_smuber = False
num_draws = 30

In [5]:
raw_data = pd.read_csv('../R/data/microcredit_project_data_cleaned.csv') 
print(raw_data)
data = dict()

data['site'] = raw_data["site"].to_numpy().astype('int')
data['profit'] = raw_data["profit"].to_numpy()
data['treatment'] = raw_data["treatment"].to_numpy().astype('int')
data['cat'] = raw_data["cat"].to_numpy().astype('int')

n_obs = len(data['profit'])

       site      profit  treatment  cat
0         1    0.000000          0    2
1         1  513.246492          1    3
2         1    0.000000          1    2
3         1    0.000000          0    2
4         1    0.000000          0    2
...     ...         ...        ...  ...
35298     7  -10.637987          0    1
35299     7   87.503514          0    3
35300     7   -5.736498          0    1
35301     7    0.000000          0    2
35302     7   28.722566          0    3

[35303 rows x 4 columns]


In [6]:
initial_fit_filename = \
    ('{base_dir}/python/output/' +
     'microcredit_project_advi_{num_draws}draws_smuber{use_smuber}.npz').format(
        base_dir=base_dir, num_draws=num_draws, use_smuber=use_smuber)

advi_dict = np.load(initial_fit_filename)

advi_param_pattern = paragami.get_pattern_from_json(
    str(advi_dict['advi_param_pattern_json']))
param_pattern = paragami.get_pattern_from_json(
    str(advi_dict['param_pattern_json']))
hess0 = advi_dict['hess0']
advi_params_free = advi_dict['advi_params_free']
advi_params = advi_param_pattern.fold(advi_params_free, free=True)
base_draws = advi_dict['base_draws']
prior_param_pattern = paragami.get_pattern_from_json(
    str(advi_dict['prior_param_pattern_json']))
prior_params = prior_param_pattern.fold(
    advi_dict['prior_params_flat'], free=False)

In [7]:
param_name = 'tau[0]'
change = 'sign'

#change = 'sign and significance'
reweight_filename = \
    ('{base_dir}/python/reweightings/amis_weight_' +
     '{param_name}_{change}_smuber{use_smuber}.npz').format(
        base_dir=base_dir, param_name=param_name, change=change,
        use_smuber=use_smuber)
reweight_dict = np.load(reweight_filename)
weights = reweight_dict['weights']
print(np.sum(1 - weights))

FileNotFoundError: [Errno 2] No such file or directory: '/home/rgiordan/Documents/git_repos/AdversarialInfluenceWorkbench/examples/microcredit_mixture//python/reweightings/amis_weight_tau[0]_sign_smuberFalse.npz'

In [ ]:
# Define objective
get_log_loss_free = paragami.FlattenFunctionInput(
    lambda advi_params: \
        mm_lib.get_advi_loss(advi_params, base_draws, data,
                             prior_params, param_pattern, weights),
    patterns=advi_param_pattern,
    free=True)

advi_loss_precond = paragami.PreconditionedFunction(get_log_loss_free)
advi_loss_precond.set_preconditioner_with_hessian(hessian=hess0)
objective_precond = paragami.OptimizationObjective(advi_loss_precond)
init_x = advi_loss_precond.precondition(advi_params_free)


In [ ]:
# Optimize
print('Optimizing!  -ㅅ-')
opt_time = time.time()
objective_precond.reset()
objective_precond.set_log_every(1)
opt_result = sp.optimize.minimize(
    method='trust-ncg',
    x0=init_x,
    fun=objective_precond.f,
    jac=objective_precond.grad,
    hessp=objective_precond.hessian_vector_product,
    options={'maxiter': 500})
opt_time = time.time() - opt_time

In [ ]:
print(opt_result.message)
print('Optimized!  ^-^')
print('Optimization took {} seconds'.format(opt_time))

advi_params_refit = advi_param_pattern.fold(
    advi_loss_precond.unprecondition(opt_result.x), free=True)
advi_params_refit_free = advi_param_pattern.flatten(advi_params_refit, free=True)

params_opt_refit = param_pattern.fold(advi_params['mean'], free=True)

In [ ]:
# Get the LR covariance
print('Computing Hessian')
objective = mm_lib.get_advi_objective(
    base_draws, data, prior_params, advi_param_pattern, param_pattern,
    weights=weights)

hess_time = time.time()
hess1 = objective.hessian(advi_params_refit_free)
hess_time = time.time() - hess_time
evs = np.linalg.eigvals(hess1)

In [ ]:
print(hess_time)
print(np.max(evs), np.min(evs), np.max(evs) / np.min(evs))

In [ ]:
print('Computing LR covariance')
hess_solver = vittles.solver_lib.get_dense_cholesky_solver(hess1)

get_advi_mean_free = paragami.FlattenFunctionInput(
    mm_lib.get_advi_mean, advi_param_pattern, free=True, argnums=[0])
advi_mean_free_jacobian = autograd.jacobian(
    get_advi_mean_free, argnum=0)(advi_params_refit_free, base_draws, param_pattern)
lr_cov = advi_mean_free_jacobian @ hess_solver(advi_mean_free_jacobian.T)